# N-Best 결과와 품사 태깅을 이용한 predictions.json 파일 생성하기

In [2]:
from konlpy.tag import Mecab
import json

In [3]:
mecab = Mecab()

## N-Best File 불러오기
* `open` 함수 내의 경로를 수정하여 원하는 `nbest_predictions.json` 파일 불러오기

In [4]:
with open('/opt/ml/p3-mrc-dok/code/outputs/koelectra-finetuned/nbest_predictions.json') as f:
    nbest = json.load(f)

In [14]:
nbest['mrc-1-000653'][0]

{'start_logit': 0.8646875023841858,
 'end_logit': 0.6166340708732605,
 'text': '최소',
 'probability': 0.33333060145378113}

In [15]:
mecab.pos(nbest['mrc-1-000653'][0]['text'])

[('최소', 'NNG')]

## 품사 태깅 일부 제거하기
* 한 개의 단어로 구성되어 있는 정답지 중 부사, 조사를 제거하고 그 다음으로 probability가 높은 답안을 뽑아 저장
* `invalid_tags`에 제거하고 싶은 태그 추가 가능
* 형태소 태그 참고 링크: https://sens.tistory.com/445

In [16]:
result_dict = {}
invalid_tags = ['MAG', 'MAJ', # 부사
                'JKS', 'JKC', 'JKG', 'JKO', 'JKB', 'JKV', 'JKQ', 'JX', 'JC', # 조사
               ]


for k in nbest:
    for j in range(20):
        tagged = mecab.pos(nbest[k][j]['text'])
        if len(tagged) == 1:
            _, tag = tagged[0]
            if tag in invalid_tags:
                continue
        result_dict[k] = nbest[k][j]['text']
        break

## 파일 저장
* `json` 파일 형태로 저장

In [22]:
with open('pred_remove_tags.json', "w") as writer:
    writer.write(json.dumps(result_dict, indent=4, ensure_ascii=False) + "\n")